# ECES 681 - Computer Vision

## Homework 3

### Problem 1:



Print out training losses and train and val set
accuracy as it trains. After training concludes, also make a plot of the training losses as well as the training
and validation-set accuracy of the model during training.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split


### Load CIFAR-10 Dataset
Load CIFAR-10 dataset for image classification. This dataset consists of 32 × 32 RGB images of 10 different categories.

In [ ]:
# Load the CIFAR10 dataset
transform = transforms.ToTensor()

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the training dataset into training and validation sets
train_set, val_set = random_split(train_dataset, [45000, 5000])
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Define Network Architecture
This network has two FC layers with one ReLU activation layer: input -> FC layer -> ReLU layer -> FC
layer -> scores. Write two_layer_net.ipynb function


In [ ]:
#  Define Network Architecture
class TwoLayerNet(nn.Module):
    """A two-layer fully connected neural network."""
    def __init__(self, input_dim=3072, hidden_dim=100, num_classes=10):
        """Initialize the network with input dimension, hidden layer size,
        and number of classes."""
        super().__init__()
        # Fully connected layer 1
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # ReLU activated Layer
        self.relu = nn.ReLU()
        # Fully connected layer 2
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        """Forward pass through the network."""
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        # Apply ReLU activation
        x = self.relu(x)
        return self.fc2(x)

In [ ]:
# Define a function to handle calculating the accuracy and loss of the model
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(data_loader)
    accuracy = correct / total
    return avg_loss, accuracy


### Train the Two-Layer Network with Loss and L2 Regularization
Initialize the two-layer network and the optimizer using stochastic gradient descent, implement a training loop for training.


The loss should be the sum of two terms:  
• A data loss term, which is the softmax loss between the model’s predicted scores and the ground-truth
image labels.  
• A regularization loss term, which penalizes the L2 norm of the weight matrices of all the fully-connected
layers of the model. You should not apply L2 regularization to the biases.

In [ ]:
# Initialize the model, loss function, and optimizer
model = TwoLayerNet(input_dim=32*32*3, hidden_dim=100, num_classes=10)

learning_rate = 0.01
# weight_decay parameter in torch automatically applies L2 regularization
# to weights only
weight_decay = 0.001

# Cross entropy loss deals with softmax loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
num_epochs = 20

train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward + optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluate on validation set
    val_acc = evaluate_accuracy(model, val_loader)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Val Acc: {val_acc:.4f}")


### Plot the Loss and Accuracy

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy', color='orange')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy")
plt.grid(True)

plt.tight_layout()
plt.show()


You will need to tune the hyperparameters of your model in order to improve it. Try changing the
hyperparameters of the model. You can consider changing any of the following hyperparameters:  

- num_train: The number of images to use for training
- hidden_dim: The width of the hidden layer of the model
- batch_size: The number of examples to use in each minibatch during SGD
- num_epochs: How long to train. An epoch is a single pass through the training set.  
- learning_rate: The learning rate to use for SGD  
- reg: The strength of the L2 regularization term  

You should tune the hyperparameters and train a model that achieves at least 40% on the validation
set. In your homework submission, include the loss / accuracy plot for your best model. After tuning
your model, run your best model exactly once on the test set.  
Your model should not take an excessive amount of time to train. For reference, our hyperparameter settings
achieve 42% accuracy on the validation set in less than 1 minute of training on a desktop with an Intel i9
CPU.